# Win Loss Ratio and League Points - Europe

In [1]:
import numpy as np
import pandas as pd
import requests as re
import time

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.47",
    "Accept-Language": "en-CA,en-US;q=0.9,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-59d23d98-22f7-47b2-be13-5851816103a6"
}

In [3]:
NA = pd.read_csv('../../data/first_model/raw/NA_all_data_378.csv')
print(NA.shape)
NA.head()

(378, 51)


,SERVER,MATCH_ID,BSUM1_ID,BTOP_CHAMP_ID,BSUM2_ID,BJGL_CHAMP_ID,BSUM3_ID,BMID_CHAMP_ID,BSUM4_ID,BBOT_CHAMP_ID,...,FIRST_TOWER,BTOWER_KILLS,RTOWER_KILLS,FIRST_BARON,BBARON_KILLS,RBARON_KILLS,FIRST_INHIB,BINHIB_KILLS,RINHIB_KILLS,WINNER
0,NORTH AMERICA,NA1_4396515853,_rz2B5gcGgG78Ce-qPHbK_BjSGuAYXVbU5HehuD69QhJE2I,50,RWNELKaFziEInyoYrBUt2CAKeHxIi_1S71GwvUdr__CJErI,200,WTTjHK7BiVw0fYiDWcPfuV248NITM-7Yvz2t1KqtYjEJzjA,38,F8TVH5gwahG6ByWfv5nt6xmqGItJAOHTP_vRj6R_kYIuDn...,119,...,BLUE,7,0,NONE,0,0,BLUE,1,0,BLUE
1,NORTH AMERICA,NA1_4396540388,r0VX6FfhshlR1S_yFNnv3FZ24b37EPCXZ0X5PUpu82Mqncs,266,qxox9oRsNSZttfe03EJKZgf5_YV7eqYl5SbsSuioeY0zlF8,9,-mhcp2urSsr5scOkFeXXMkzMhzrrJSZ1dWH4z3xLgjlukls,8,F8TVH5gwahG6ByWfv5nt6xmqGItJAOHTP_vRj6R_kYIuDn...,119,...,BLUE,8,4,BLUE,1,0,BLUE,1,0,BLUE
2,NORTH AMERICA,NA1_4396495283,g96u4Zu60wNHE_ghxcPDuC_Mc9ouc3m48Mx_NLdZ1FMdfxg,266,Dr8A8hpa5aBnlkY4qMoo8d40qEcqOIIlfc0C3ojR4JGkJmQ,200,F8TVH5gwahG6ByWfv5nt6xmqGItJAOHTP_vRj6R_kYIuDn...,1,V-vrG5r7dzvV3vppV-vDVZXsFAE0jMQWCIMJPYom6RjAhS...,360,...,BLUE,9,5,BLUE,1,0,BLUE,1,0,BLUE
3,NORTH AMERICA,NA1_4396437949,Mj8iPiDSI3xHQO_1gDFsGz5uf6dwY7vHZla08rzzMuRmHYc,27,H9lyL4tk2OM82hgYSU36Dr-MlVdpGFY7OBQ45cQSnP0jKzNh,421,36g0ZTyQO7y54a6u-u2oAR0lAWoIrU7rL_V65ipeEyautWY,74,DWXk985dJPD6ywHdP9yIVJQe7kGLMVb3c502hFDTLGIaJyU,119,...,BLUE,3,7,RED,0,1,RED,0,2,RED
4,NORTH AMERICA,NA1_4396434056,daODRa4nnhTu0q1-eycLcvTqE_eG9-knakaOpTdDwymGsD0,516,MxCz3g6oBXQubGlvJnMTvdaJ73L6J9CIlYgzoj9nOY_SXuA,131,d1taUIColLPNBQdrVyuo-LdG80RYmo6Bqk_qU5gPsYGsLQs,104,xL-gHga9xDBG2avQT56HZlPRLQUKX2OkvJkfsOtr29S2mr...,523,...,BLUE,4,11,RED,0,1,RED,0,2,RED


In [4]:
NA1 = NA[:100]
NA2 = NA[100:200]
NA3 = NA[200:300]
NA4 = NA[300:]

In [5]:
'''
This function uses the champion mastery api to get the champion mastery for a specific champion and summoner.

Region can be: euw1, na1, kr

Summoner input can be:
-  BSUM1 - BSUM5
-  RSUM1 - RSUM5

SummonerID input can be:
-  BSUM1_ID - BSUM5_ID
-  RSUM1_ID - RSUM5_ID
'''

def summoner_stats(df, server, summoner, summonerID):

    stats_df = pd.DataFrame()
    wins_column = summoner + '_WINS'
    losses_column = summoner + '_LOSSES'
    league_points_column = summoner + '_LEAGUE_POINTS'

    wins_list = []
    losses_list = []
    league_points_list = []

    for i in range(len(df)):
        summoner_id = df.iloc[i][summonerID]
        try: 
            url = 'https://'+ server + '.api.riotgames.com/lol/league/v4/entries/by-summoner/' + summoner_id
            res = re.get(url, headers=headers).json()

            wins = res[0]['wins']
            wins_list.append(wins)

            losses = res[0]['losses']
            losses_list.append(losses)

            leaguePoints = res[0]['leaguePoints']
            league_points_list.append(leaguePoints)
        
        except:
            wins_list.append(np.nan)
            losses_list.append(np.nan)
            league_points_list.append(np.nan)
            pass

    stats_df[wins_column] = wins_list
    stats_df[losses_column] = losses_list
    stats_df[league_points_column] = league_points_list     

    return stats_df

## Blue Team

### SUMMONER 1 - TOP

In [6]:
BTOP1 = summoner_stats(NA1, 'na1', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP2 = summoner_stats(NA2, 'na1', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP3 = summoner_stats(NA3, 'na1', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP4 = summoner_stats(NA4, 'na1', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP_stats = pd.concat([BTOP1, BTOP2, BTOP3, BTOP4], axis=0)
print(BTOP_stats.shape)
BTOP_stats.head()

(378, 3)


,BTOP_WINS,BTOP_LOSSES,BTOP_LEAGUE_POINTS
0,371.0,360.0,419.0
1,32.0,40.0,100.0
2,217.0,192.0,833.0
3,327.0,306.0,442.0
4,767.0,733.0,580.0


In [11]:
BTOP_stats.isna().any()

BTOP_WINS             True
BTOP_LOSSES           True
BTOP_LEAGUE_POINTS    True
dtype: bool

### SUMMONER 2 - JGL

In [12]:
BJGL1 = summoner_stats(NA1, 'na1', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL2 = summoner_stats(NA2, 'na1', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL3 = summoner_stats(NA3, 'na1', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL4 = summoner_stats(NA4, 'na1', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL_stats = pd.concat([BJGL1, BJGL2, BJGL3, BJGL4], axis=0)
print(BJGL_stats.shape)
BJGL_stats.head()

(378, 3)


,BJGL_WINS,BJGL_LOSSES,BJGL_LEAGUE_POINTS
0,158.0,132.0,633.0
1,54.0,35.0,75.0
2,32.0,13.0,88.0
3,17.0,14.0,100.0
4,82.0,64.0,209.0


In [13]:
BJGL_stats.isna().any()

BJGL_WINS             True
BJGL_LOSSES           True
BJGL_LEAGUE_POINTS    True
dtype: bool

### SUMMONER 3 - MID

In [14]:
BMID1 = summoner_stats(NA1, 'na1', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID2 = summoner_stats(NA2, 'na1', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID3 = summoner_stats(NA3, 'na1', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID4 = summoner_stats(NA4, 'na1', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID_stats = pd.concat([BMID1, BMID2, BMID3, BMID4], axis=0)
print(BMID_stats.shape)
BMID_stats.head()

(378, 3)


,BMID_WINS,BMID_LOSSES,BMID_LEAGUE_POINTS
0,12.0,19.0,7.0
1,500.0,449.0,684.0
2,813.0,771.0,562.0
3,25.0,13.0,100.0
4,941.0,925.0,312.0


In [15]:
BMID_stats.isna().any()

BMID_WINS             True
BMID_LOSSES           True
BMID_LEAGUE_POINTS    True
dtype: bool

### SUMMONER 4 - BOT

In [16]:
BBOT1 = summoner_stats(NA1, 'na1', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT2 = summoner_stats(NA2, 'na1', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT3 = summoner_stats(NA3, 'na1', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT4 = summoner_stats(NA4, 'na1', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT_stats = pd.concat([BBOT1, BBOT2, BBOT3, BBOT4], axis=0)
print(BBOT_stats.shape)
BBOT_stats.head()

(378, 3)


,BBOT_WINS,BBOT_LOSSES,BBOT_LEAGUE_POINTS
0,813.0,771.0,562.0
1,813.0,771.0,562.0
2,115.0,76.0,1034.0
3,326.0,312.0,450.0
4,176.0,159.0,552.0


In [17]:
BBOT_stats.isna().any()

BBOT_WINS             True
BBOT_LOSSES           True
BBOT_LEAGUE_POINTS    True
dtype: bool

### SUMMONER 5 - SUP

In [18]:
BSUP1 = summoner_stats(NA1, 'na1', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP2 = summoner_stats(NA2, 'na1', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP3 = summoner_stats(NA3, 'na1', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP4 = summoner_stats(NA4, 'na1', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP_stats = pd.concat([BSUP1, BSUP2, BSUP3, BSUP4], axis=0)
print(BSUP_stats.shape)
BSUP_stats.head()

(378, 3)


,BSUP_WINS,BSUP_LOSSES,BSUP_LEAGUE_POINTS
0,454.0,424.0,438.0
1,15.0,8.0,61.0
2,161.0,112.0,1189.0
3,176.0,159.0,552.0
4,8.0,9.0,39.0


## Red Team

### Summoner 1 - TOP

In [19]:
RTOP1 = summoner_stats(NA1, 'na1', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP2 = summoner_stats(NA2, 'na1', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP3 = summoner_stats(NA3, 'na1', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP4 = summoner_stats(NA4, 'na1', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP_stats = pd.concat([RTOP1, RTOP2, RTOP3, RTOP4], axis=0)
print(RTOP_stats.shape)
RTOP_stats.head()

(378, 3)


,RTOP_WINS,RTOP_LOSSES,RTOP_LEAGUE_POINTS
0,10.0,7.0,42.0
1,457.0,430.0,544.0
2,672.0,643.0,836.0
3,892.0,876.0,374.0
4,892.0,876.0,374.0


In [20]:
RTOP_stats.isna().any()

RTOP_WINS             True
RTOP_LOSSES           True
RTOP_LEAGUE_POINTS    True
dtype: bool

### Summoner 2 - JGL

In [21]:
RJGL1 = summoner_stats(NA1, 'na1', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL2 = summoner_stats(NA2, 'na1', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL3 = summoner_stats(NA3, 'na1', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL4 = summoner_stats(NA4, 'na1', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL_stats = pd.concat([RJGL1, RJGL2, RJGL3, RJGL4], axis=0)
print(RJGL_stats.shape)
RJGL_stats.head()

(378, 3)


,RJGL_WINS,RJGL_LOSSES,RJGL_LEAGUE_POINTS
0,60.0,48.0,1.0
1,119.0,98.0,490.0
2,511.0,481.0,752.0
3,489.0,455.0,814.0
4,489.0,455.0,814.0


In [22]:
RJGL_stats.isna().any()

RJGL_WINS             True
RJGL_LOSSES           True
RJGL_LEAGUE_POINTS    True
dtype: bool

### Summoner 3 - MID

In [23]:
RMID1 = summoner_stats(NA1, 'na1', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID2 = summoner_stats(NA2, 'na1', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID3 = summoner_stats(NA3, 'na1', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID4 = summoner_stats(NA4, 'na1', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID_stats = pd.concat([RMID1, RMID2, RMID3, RMID4], axis=0)
print(RMID_stats.shape)
RMID_stats.head()

(378, 3)


,RMID_WINS,RMID_LOSSES,RMID_LEAGUE_POINTS
0,500.0,449.0,684.0
1,226.0,186.0,382.0
2,537.0,473.0,1506.0
3,42.0,13.0,40.0
4,676.0,650.0,616.0


In [24]:
RMID_stats.isna().any()

RMID_WINS             True
RMID_LOSSES           True
RMID_LEAGUE_POINTS    True
dtype: bool

### Summoner 4 - BOT

In [25]:
RBOT1 = summoner_stats(NA1, 'na1', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT2 = summoner_stats(NA2, 'na1', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT3 = summoner_stats(NA3, 'na1', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT4 = summoner_stats(NA4, 'na1', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT_stats = pd.concat([RBOT1, RBOT2, RBOT3, RBOT4], axis=0)
print(RBOT_stats.shape)
RBOT_stats.head()

(378, 3)


,RBOT_WINS,RBOT_LOSSES,RBOT_LEAGUE_POINTS
0,103.0,93.0,327.0
1,789.0,771.0,691.0
2,79.0,50.0,16.0
3,813.0,771.0,562.0
4,813.0,771.0,562.0


In [26]:
RBOT_stats.isna().any()

RBOT_WINS             True
RBOT_LOSSES           True
RBOT_LEAGUE_POINTS    True
dtype: bool

### Summoner 5 - SUP

In [27]:
RSUP1 = summoner_stats(NA1, 'na1', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP2 = summoner_stats(NA2, 'na1', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP3 = summoner_stats(NA3, 'na1', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP4 = summoner_stats(NA4, 'na1', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP_stats = pd.concat([RSUP1, RSUP2, RSUP3, RSUP4], axis=0)
print(RSUP_stats.shape)
RSUP_stats.head()

(378, 3)


,RSUP_WINS,RSUP_LOSSES,RSUP_LEAGUE_POINTS
0,19.0,16.0,24.0
1,69.0,39.0,257.0
2,142.0,90.0,696.0
3,25.0,28.0,12.0
4,69.0,53.0,328.0


In [28]:
RSUP_stats.isna().any()

RSUP_WINS             True
RSUP_LOSSES           True
RSUP_LEAGUE_POINTS    True
dtype: bool

### Combine into one DataFrame

In [29]:
NA_stats = pd.concat([BTOP_stats, BJGL_stats, BMID_stats, BBOT_stats, BSUP_stats, RTOP_stats, RJGL_stats, RMID_stats, RBOT_stats, RSUP_stats], axis=1)
print(NA_stats.shape)
NA_stats.head()

(378, 30)


,BTOP_WINS,BTOP_LOSSES,BTOP_LEAGUE_POINTS,BJGL_WINS,BJGL_LOSSES,BJGL_LEAGUE_POINTS,BMID_WINS,BMID_LOSSES,BMID_LEAGUE_POINTS,BBOT_WINS,...,RJGL_LEAGUE_POINTS,RMID_WINS,RMID_LOSSES,RMID_LEAGUE_POINTS,RBOT_WINS,RBOT_LOSSES,RBOT_LEAGUE_POINTS,RSUP_WINS,RSUP_LOSSES,RSUP_LEAGUE_POINTS
0,371.0,360.0,419.0,158.0,132.0,633.0,12.0,19.0,7.0,813.0,...,1.0,500.0,449.0,684.0,103.0,93.0,327.0,19.0,16.0,24.0
1,32.0,40.0,100.0,54.0,35.0,75.0,500.0,449.0,684.0,813.0,...,490.0,226.0,186.0,382.0,789.0,771.0,691.0,69.0,39.0,257.0
2,217.0,192.0,833.0,32.0,13.0,88.0,813.0,771.0,562.0,115.0,...,752.0,537.0,473.0,1506.0,79.0,50.0,16.0,142.0,90.0,696.0
3,327.0,306.0,442.0,17.0,14.0,100.0,25.0,13.0,100.0,326.0,...,814.0,42.0,13.0,40.0,813.0,771.0,562.0,25.0,28.0,12.0
4,767.0,733.0,580.0,82.0,64.0,209.0,941.0,925.0,312.0,176.0,...,814.0,676.0,650.0,616.0,813.0,771.0,562.0,69.0,53.0,328.0


In [92]:
NA_stats.to_csv('EU_stats_378.csv', index=False)